In [1]:
import netCDF4 as nc
import numpy as np
import xarray as xr
import pandas as pd
from glob import glob

In [ ]:
ds = xr.open_mfdataset('/content/drive/MyDrive/data/methane_old/*.nc', concat_dim=["sounding_dim"], combine="nested")
ds.to_netcdf('/content/drive/MyDrive/data/test/methane_L.nc')

In [ ]:
my_nc_file = '/content/drive/MyDrive/data/CO2/20120601-C3S-L2_GHG-GHG_PRODUCTS-MERGED-MERGED-EMMA-DAILY-v4.3.nc'
ds = nc.Dataset(my_nc_file)
for dim in ds.dimensions.values():
    print(dim)

<class 'netCDF4._netCDF4.Dimension'>: name = 'sounding_dim', size = 489
<class 'netCDF4._netCDF4.Dimension'>: name = 'layer_dim', size = 10
<class 'netCDF4._netCDF4.Dimension'>: name = 'level_dim', size = 11


In [ ]:
my_nc_file = "/content/drive/MyDrive/data/CO2/20120601-C3S-L2_GHG-GHG_PRODUCTS-MERGED-MERGED-EMMA-DAILY-v4.3.nc"
ds = xr.open_mfdataset(my_nc_file)
m_data = ds.to_dataframe().reset_index(drop=True)
m_data             

,longitude,latitude,time,solar_zenith_angle,sensor_zenith_angle,xco2_quality_flag,xco2,xco2_uncertainty,co2_profile_apriori,pressure_levels,pressure_weight,xco2_averaging_kernel,median_processor_id,median_uncertainty,median_uncertainty_se,median_uncertainty_ex,xco2_accuracy,contributing_algorithms
0,-162.774109,67.429771,2012-06-01 00:59:10.368994560,0.0,0.0,0,396.676880,1.911436,398.717560,1003.510498,0.1,1.035140,2000,0.723221,0.132753,0.283400,0.407804,6
1,-162.775452,67.430260,2012-06-01 00:59:14.977981696,0.0,0.0,0,395.063690,1.931680,398.717560,1003.513184,0.1,1.023195,2000,0.723221,0.132753,0.283400,0.407804,6
2,-162.778152,67.429016,2012-06-01 00:59:19.588015104,0.0,0.0,0,395.975250,1.928886,398.717590,1003.542847,0.1,1.024706,2000,0.723221,0.132753,0.283400,0.407804,6
3,-157.362122,66.286560,2012-06-01 00:59:26.199003136,0.0,0.0,0,397.843384,1.999521,398.395599,923.216797,0.1,0.986646,5,0.849926,0.134209,0.285535,0.504706,6
4,-157.364090,66.284111,2012-06-01 00:59:30.806019072,0.0,0.0,0,396.185028,1.973080,398.395599,924.037720,0.1,0.986327,5,0.849926,0.134209,0.285535,0.504706,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28485,-147.674637,5.661206,2012-06-01 22:33:07.660016128,0.0,0.0,0,393.313782,1.913724,389.774597,0.000000,0.1,0.688884,2000,0.861378,0.262414,0.517967,0.407804,5
28486,-147.675644,5.664542,2012-06-01 22:33:12.269003264,0.0,0.0,0,393.661804,1.909479,389.774597,0.000000,0.1,0.699071,2000,0.861378,0.262414,0.517967,0.407804,5
28487,-154.329590,65.727356,2012-06-01 23:54:40.264015360,0.0,0.0,0,393.266205,1.891128,385.096741,0.000000,0.1,0.544168,5,0.849926,0.134209,0.285535,0.504706,6
28488,-154.328979,65.728973,2012-06-01 23:54:44.875014400,0.0,0.0,0,395.393097,1.894273,385.096741,0.000000,0.1,0.544335,5,0.849926,0.134209,0.285535,0.504706,6


In [10]:
def concat_dataframe(paths):  
  ret_df = pd.DataFrame()
  for p in paths:    
    ds = xr.open_dataset(p)
    m_data = ds.to_dataframe().reset_index(drop=True)
    m_data = m_data[["time","xco2"]]
    m_data['time'] = m_data['time'].dt.date
    m_data_mean = m_data.groupby("time").mean()
    ds.close()
    if ret_df.empty:
      ret_df = m_data_mean 
    else:
      ret_df = pd.concat([ret_df, m_data_mean])   
  return ret_df


paths = sorted(glob('/content/drive/MyDrive/data/CO2_old/*.nc'))
result_df = concat_dataframe(paths)
result_df.to_csv("/content/drive/MyDrive/data/CO2_L.csv")
result_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

,xco2
time,
2003-01-08,373.659576
2003-01-10,375.680817
2003-01-11,374.629547
2003-01-12,375.125793
2003-01-13,374.611023
...,...
2020-06-26,411.219238
2020-06-27,412.029358
2020-06-28,411.490417
